# Introduksjon til prompt engineering
Prompt engineering er prosessen med å designe og optimalisere oppgaver for oppgaver innen naturlig språkbehandling. Det handler om å velge riktige oppgaver, justere parametere, og evaluere ytelsen. Prompt engineering er avgjørende for å oppnå høy nøyaktighet og effektivitet i NLP-modeller. I denne delen skal vi se på grunnleggende prinsipper for prompt engineering ved å bruke OpenAI-modellene til utforskning.


### Øvelse 1: Tokenisering
Utforsk tokenisering med tiktoken, en åpen kildekode og rask tokenizer fra OpenAI
Se [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst) for flere eksempler.


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### Øvelse 2: Sjekk oppsettet for Github Models-nøkkel

Kjør koden under for å sjekke at Github Models-endepunktet ditt er satt opp riktig. Koden tester bare en enkel prompt og sjekker at svaret er som forventet. Inndataen `oh say can you see` bør fullføres med noe sånt som `by the dawn's early light..`


In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt, client, model_name)
print(response)


That line is the opening lyric of "The Star-Spangled Banner," the national anthem of the United States, written by Francis Scott Key. If you'd like more information or analysis, feel free to ask!


### Øvelse 3: Oppdiktede svar
Undersøk hva som skjer når du ber LLM-en om å gi svar på en oppgave om et tema som kanskje ikke eksisterer, eller om temaer den kanskje ikke kjenner til fordi det var utenfor dens forhåndstrente datasett (mer nylig). Se hvordan svaret endrer seg hvis du prøver en annen oppgave, eller en annen modell.


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

### Øvelse 4: Instruksjonsbasert
Bruk variabelen "text" for å angi hovedinnholdet
og variabelen "prompt" for å gi en instruksjon knyttet til det hovedinnholdet.

Her ber vi modellen om å oppsummere teksten for en elev på andre trinn


In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt, client, model_name)
print(response)

Jupiter is the fifth planet from the Sun and the biggest one in our Solar System. It's made of gas and is much bigger than all the other planets put together! You can see Jupiter in the night sky because it's very bright. People have noticed it for a really long time and named it after a Roman god.


### Øvelse 5: Kompleks prompt
Prøv en forespørsel som har system-, bruker- og assistentmeldinger
Systemet setter assistentens kontekst
Bruker- og assistentmeldinger gir kontekst for en samtale med flere runder

Legg merke til hvordan assistentens personlighet er satt til "sarkastisk" i systemkonteksten.
Prøv å bruke en annen personlighet i konteksten. Eller prøv en annen serie med inn-/utmeldinger


In [5]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, you mean the famous 2020 World Series that wasn’t in a regular location? That was the year they played in the glamorous Arlington, Texas, at Globe Life Field.


### Øvelse: Utforsk intuisjonen din
Eksemplene ovenfor gir deg mønstre du kan bruke for å lage nye oppgaver (enkle, komplekse, instruksjoner osv.) – prøv å lage andre øvelser for å utforske noen av de andre ideene vi har snakket om, som eksempler, hint og mer.



---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi tilstreber nøyaktighet, vær oppmerksom på at automatiske oversettelser kan inneholde feil eller unøyaktigheter. Det opprinnelige dokumentet på sitt originale språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell, menneskelig oversettelse. Vi er ikke ansvarlige for eventuelle misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
